In [24]:
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.linear_model import SGDRegressor
from scipy.stats import uniform
import pandas as pd
import numpy as np

In [25]:
df = pd.read_csv('data/train_processed.csv')
X_train, y_train = df.drop(columns=['SalePrice', 'Id']), df['SalePrice']
X_test = pd.read_csv('data/test-processed.csv')

In [26]:
params = {
    'loss': ['squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
    'penalty': ['l2', 'l1', 'elasticnet', None],
    'alpha': uniform(0.0001, 0.1),
    'l1_ratio': uniform(0, 1)
}

In [27]:
sgd = SGDRegressor()
random_search = RandomizedSearchCV(
    estimator=sgd,
    param_distributions=params,
    n_iter=50,
    scoring='neg_mean_squared_error',
    cv=5
)
random_search.fit(X_train, y_train)

SGDRegressor()

In [28]:
best_model = random_search.best_estimator_
scores = cross_val_score(best_model, X_train, y_train, scoring='neg_mean_squared_error', cv=10)
scores = np.sqrt(-scores)
print(scores.mean())
scores

30670.470598196687


array([20714.94210957, 25630.12032821, 25001.82381656, 38884.90991532,
       34139.93147346, 29404.33621657, 27494.45153617, 24429.40022899,
       56223.28858226, 24781.50177485])

In [30]:
X_test.fillna(method='ffill', inplace=True)
predictions = pd.DataFrame({'Id': X_test['Id'], 'SalePrice': best_model.predict(X_test.drop(columns=['Id']))})
predictions.set_index('Id', inplace=True)
predictions.to_csv('data/predictions/sgd.csv')